In [1]:
!pip install --upgrade google-api-python-client
!pip install pandas_gbq
!pip install mpu
import sqlite3
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
from scipy.spatial import distance
import numpy as np
import mpu

Requirement already up-to-date: google-api-python-client in c:\users\celherme1\anaconda3\lib\site-packages (1.7.8)


## Weather

In [2]:
# Get a dataframe of all the cities at hand, along with their latitude and longitude
conn = sqlite3.connect('../cityDB.sqlite')
cities = pd.read_sql_query("SELECT cid, city, state, lat, lng  FROM cities;", conn)

# Get the coordinates for the cities
cities['coordinates'] = cities.apply(lambda x: (x['lat'], x['lng']), axis=1)

In [3]:
# Get a list of all the stations in the US from NOAA data publicly available on BigQuery
credentials = service_account.Credentials.from_service_account_file('bigquery_key.json')
stations_query = ("""SELECT *
FROM `bigquery-public-data.noaa_gsod.stations`
WHERE country = 'US'""")
stations = pandas_gbq.read_gbq(stations_query,
                               project_id="dva-destination-recommender",
                               credentials=credentials, dialect='standard')

# Remove stations with null values
stations.dropna(axis=0, inplace=True)

# Get the coordinates for the stations
stations['coordinates'] = stations.apply(lambda x: (x.lat, x.lon), axis=1)

In [4]:
# Get the monthly weather over the past 5 years for all stations
weather_query = ("""
SELECT stn, wban, mo as month, AVG(temp) as avg_temp, AVG(prcp) as avg_prcp, SUM(CAST(fog AS int64))/5 as foggy_days, 
SUM(CAST(rain_drizzle as int64))/5 as rainy_days, 
SUM(CAST(snow_ice_pellets as int64))/5 as snow_days, 
SUM(CAST(thunder as int64))/5 as stormy_days
FROM
(SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2018`
UNION ALL
SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2017`
UNION ALL
SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2016`
UNION ALL
SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2015`
UNION ALL
SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2014`) a
 GROUP BY 1,2,3""")

weather = pandas_gbq.read_gbq(weather_query,
                               project_id="dva-destination-recommender",
                               credentials=credentials, dialect='standard')

In [5]:
# Only keep the stations for which we have weather info
stations_with_weather = weather.merge(stations, how='inner', left_on=['stn', 'wban'], right_on=['usaf', 'wban'])

In [6]:
def find_closest_station(city_coord, stations):
    """
    INPUTS:
    city_coord (tuple): a tuple with the coordinates for the city
    stations (Pandas DataFrame): a Pandas DataFrame with station information

    OUTPUT:
    ids (int): the station ID for the closest station to the city, both usaf and wban
    """
    # Initialize a dictionary to hold the distance with each station coordinates
    station_dict = {}
    
    for station in stations['coordinates'].unique().tolist():
        station_dict[station] = mpu.haversine_distance(city_coord, station)
        
    closest_station = sorted(station_dict.items(), key=lambda x: x[1])[0][0]
    usaf = stations.loc[stations['coordinates'] == closest_station, 'usaf'].iloc[0]
    wban = stations.loc[stations['coordinates'] == closest_station, 'wban'].iloc[0]

    ids = str(usaf) + '|' + str(wban)

    return ids

In [7]:
# Find the closest station for each city in the dataframe
closest_stations = []
for row_num in range(len(cities)):
    city_coord = cities.iloc[row_num].coordinates
    closest_stations.append(find_closest_station(city_coord, stations_with_weather))

In [8]:
cities['closest_station'] = closest_stations

In [9]:
cities[['closest_station_usaf', 'closest_station_wban']] = cities['closest_station'].str.split('|',expand=True)

In [10]:
# Append weather information to the cities
city_monthly_weather = cities.merge(stations_with_weather, how='left', left_on=['closest_station_wban', 'closest_station_usaf'],
                                   right_on=['wban', 'stn'])

In [11]:
# Drop unecessary columns and columns with unsupported types in sqlite
city_monthly_weather.drop(labels=['usaf', 'name', 'country', 'state_y', 'coordinates_x', 
                                 'call', 'lat_y', 'lon', 'elev', 'begin', 'end', 'coordinates_y', 'stn', 'wban'], 
                          inplace=True, axis=1)

In [12]:
# Rename columns
city_monthly_weather.rename(axis=1, mapper={'state_x': 'state', 'lat_x': 'lat', 'lng': 'lon'}, inplace=True)

In [13]:
# Store to database
city_monthly_weather.to_sql('city_weather', conn, if_exists='replace')

## Airport

In [118]:
airport_info = pd.read_csv('../AirportData/airports_info.txt', names=['openflights_airport_id', 'name', 'city', 'country',
                                                         'iata_code', 'icao_code', 'latitude',
                                                         'longitude', 'altitude', 'tz', 'dst', 'olson_tz',
                                                         'type', 'source'])
airport_info['coordinates'] = airport_info.apply(lambda x: (x.latitude, x.longitude), axis=1)

In [119]:
airport_info.head()

,openflights_airport_id,name,city,country,iata_code,icao_code,latitude,longitude,altitude,tz,dst,olson_tz,type,source,coordinates
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,"(-6.0816898345900015, 145.391998291)"
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports,"(-5.20707988739, 145.789001465)"
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports,"(-5.826789855957031, 144.29600524902344)"
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports,"(-6.569803, 146.725977)"
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports,"(-9.443380355834961, 147.22000122070312)"


In [116]:
origin = cities.iloc[2].coordinates

In [26]:
airport_dict = {}
airport_coordinates = airport_info.coordinates.tolist()
for airport in airport_coordinates:
        airport_dict[airport] = mpu.haversine_distance(origin, airport)

In [35]:
closest_airports = sorted(airport_dict.items(), key=lambda x: x[1])[0:3]

In [38]:
[u for u,v in closest_airports if v <= 40]

[(40.71260070800781, -73.99960327148438),
 (40.77719879, -73.87259674),
 (40.69250106811523, -74.168701171875)]

In [169]:
def find_closest_airport(origin, airport_df, n_airports, radius):
    """
    :param origin: a tuple with the latitude and longitude of the origin (either a city or an individual)
    :param airport_df: a Pandas dataframe containing airport information, at least iata_code, latitude and longitude
    :param n_airports: the number of airports we want to return, pad with NULL's if not enough airports
    :param radius: the maximum distance, in kilometers

    :return: closest_airports_code: a list of n airport codes as the closest airports
    """
    # Extract the top n, only if less than radius
    
    # Initialize a dictionary to hold the distance with each airport
    airport_dict = {}
    
    # Extract coordinates of each airport
    airport_coordinates = airport_df.coordinates.tolist()
    
    # Get the distance with each airport coordinates
    for airport in airport_coordinates:
        airport_dict[airport] = mpu.haversine_distance(origin, airport)
    
    # Extract the n closest airports
    closest_airports = sorted(airport_dict.items(), key=lambda x: x[1])[0:n_airports]
    
    # Check the distance is less than the desired radius
    closest_airports_radius = [u for u,v in closest_airports if v <= radius]
    
    # Get the airport codes for all the coordinates in the list
    airport_codes = []
    for close_airport in closest_airports_radius:
        airport_codes.append(airport_df.loc[airport_df.coordinates == close_airport, 'iata_code'].values[0])
    
    if len(airport_codes) < n_airports:
        return airport_codes + (n_airports-len(airport_codes))*['']
    
    else:
        return airport_codes

In [166]:
find_closest_airport(origin, airport_info, n_airports=5, radius=70)

['GYY', 'CGX', 'MDW', 'ORD', '']

In [81]:
airport_price = pd.read_sql_query("SELECT *  FROM fligh_price_history;", conn)

In [104]:
airports_with_price = airport_price.src.unique().tolist()

In [122]:
airport_info = airport_info.loc[airport_info.iata_code.isin(airports_with_price),]

In [157]:
find_closest_airport(origin, airport_info, n_airports=5, radius=30)

['GYY', 'CGX', 'MDW', '', '']

In [154]:
city_airport = []

In [170]:
for i in range(len(cities)):
    city = cities.iloc[i]
    city_id = city.cid
    city_name = city.city
    state = city.state
    closest_airports = find_closest_airport(city.coordinates, airport_info, n_airports=5, radius=60)
    print(closest_airports)
    city_airport.append({'city_id': city_id, 'city_name': city_name, 'state': state, 
                         'closest_airport': closest_airports[0], 'second_closest_airport': closest_airports[1],
                        'third_closest_airport': closest_airports[2]})
city_airport = pd.DataFrame(city_airport)

['LGA', 'EWR', 'JFK', '', '']
['LAX', 'BUR', 'LGB', 'SNA', 'ONT']
['GYY', 'CGX', 'MDW', 'ORD', '']
['HOU', 'IAH', '', '', '']
['PHX', '', '', '', '']
['PHL', 'TTN', '', '', '']
['SAT', '', '', '', '']
['', '', '', '', '']
['SAN', 'CLD', '', '', '']
['SJC', 'OAK', 'SFO', '', '']
['DTW', '', '', '', '']
['SFO', 'OAK', 'CCR', '', '']
['JAX', '', '', '', '']
['IND', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['CLT', '', '', '', '']
['MEM', '', '', '', '']
['BWI', '', '', '', '']
['ELP', '', '', '', '']
['BOS', '', '', '', '']
['MKE', '', '', '', '']
['DEN', 'WBU', '', '', '']
['SEA', '', '', '', '']
['BNA', '', '', '', '']
['', '', '', '', '']
['SBN', 'GYY', '', '', '']
['', '', '', '', '']
['SDF', '', '', '', '']
['OKC', '', '', '', '']
['TUS', '', '', '', '']
['GYY', 'MDW', 'CGX', '', '']
['ABQ', '', '', '', '']
['FAT', 'VIS', '', '', '']
['SMF', '', '', '', '']
['LGB', 'LAX', 'SNA', 'BUR', '']
['PHX', '', '', '', '']
['MCI', '', '', '', '']
['OMA', ''

['LAX', 'BUR', 'LGB', '', '']
['ATL', '', '', '', '']
['BOS', '', '', '', '']
['MHT', 'BOS', '', '', '']
['DTW', '', '', '', '']
['SMX', 'SBP', '', '', '']
['GEG', '', '', '', '']
['SBA', '', '', '', '']
['BUR', 'LGB', 'LAX', 'ONT', 'SNA']
['WBU', 'DEN', '', '', '']
['ONT', '', '', '', '']
['FLL', 'PBI', '', '', '']
['CSG', '', '', '', '']
['IND', '', '', '', '']
['PVD', '', '', '', '']
['PSP', '', '', '', '']
['SMF', '', '', '', '']
['DAB', 'ORL', 'MCO', '', '']
['SEA', '', '', '', '']
['LAX', 'LGB', 'BUR', 'SNA', '']
['DLH', '', '', '', '']
['MCI', '', '', '', '']
['CLL', '', '', '', '']
['YKM', '', '', '', '']
['SEA', '', '', '', '']
['ONT', 'SNA', 'LGB', '', '']
['FLL', '', '', '', '']
['HVN', 'LGA', '', '', '']
['ONT', 'SNA', 'LGB', '', '']
['HOU', 'IAH', '', '', '']
['TTN', 'PHL', '', '', '']
['SLC', '', '', '', '']
['SUX', '', '', '', '']
['ATL', '', '', '', '']
['ORF', 'PHF', '', '', '']
['LGB', 'LAX', 'SNA', 'BUR', 'ONT']
['MKE', '', '', '', '']
['BOS', 'ORH', '', '', '']
['PH

['MDW', 'CGX', 'GYY', 'ORD', '']
['BFL', '', '', '', '']
['AUS', '', '', '', '']
['SMF', '', '', '', '']
['CCR', 'STS', 'OAK', 'SFO', '']
['SBN', '', '', '', '']
['OKC', '', '', '', '']
['', '', '', '', '']
['BWI', '', '', '', '']
['RDU', '', '', '', '']
['HVN', '', '', '', '']
['SRQ', '', '', '', '']
['MHK', '', '', '', '']
['PSP', '', '', '', '']
['BMI', '', '', '', '']
['AZO', '', '', '', '']
['SEA', '', '', '', '']
['PIB', '', '', '', '']
['DTW', '', '', '', '']
['VIS', '', '', '', '']
['CLE', '', '', '', '']
['DAY', 'CVG', '', '', '']
['BUF', '', '', '', '']
['BOS', '', '', '', '']
['LGB', 'SNA', 'LAX', 'BUR', 'ONT']
['GFK', '', '', '', '']
['MLU', '', '', '', '']
['EWR', 'TTN', 'LGA', 'JFK', '']
['SLC', '', '', '', '']
['SLE', 'EUG', '', '', '']
['ONT', '', '', '', '']
['CLE', '', '', '', '']
['ONT', 'SNA', '', '', '']
['LSE', '', '', '', '']
['PSP', '', '', '', '']
['GSO', '', '', '', '']
['ORL', 'MCO', 'DAB', '', '']
['BLV', '', '', '', '']
['', '', '', '', '']
['MRY', 'SJC', '

['BUR', 'LAX', 'LGB', '', '']
['TPA', '', '', '', '']
['', '', '', '', '']
['SLE', '', '', '', '']
['', '', '', '', '']
['GLH', '', '', '', '']
['ALM', '', '', '', '']
['SNA', 'CLD', 'LGB', '', '']
['JFK', 'LGA', 'EWR', '', '']
['SLC', '', '', '', '']
['', '', '', '', '']
['XNA', 'FYV', '', '', '']
['PBI', '', '', '', '']
['MKE', '', '', '', '']
['ONT', 'SNA', 'LGB', 'BUR', '']
['MOT', '', '', '', '']
['SLC', '', '', '', '']
['DTW', '', '', '', '']
['BDL', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['CCR', 'OAK', 'SFO', '', '']
['FAI', '', '', '', '']
['AUG', 'PWM', '', '', '']
['ONT', 'LGB', 'SNA', 'BUR', 'LAX']
['MCE', '', '', '', '']
['', '', '', '', '']
['MRY', '', '', '', '']
['SNA', 'CLD', 'LGB', '', '']
['ATL', '', '', '', '']
['MCI', '', '', '', '']
['CLE', '', '', '', '']
['MKE', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['HOU', 'IAH', '', '', '']
['', '', '', '', '']
['MDW', 'ORD', 'CGX', 'GYY', '']
['HOU', 'IAH', '', '', ''

['', '', '', '', '']
['', '', '', '', '']
['ORD', 'MDW', 'CGX', '', '']
['LEX', '', '', '', '']
['ORD', 'CGX', 'MDW', 'GYY', '']
['HKY', '', '', '', '']
['MLI', '', '', '', '']
['SAT', '', '', '', '']
['', '', '', '', '']
['JAN', '', '', '', '']
['LGA', 'EWR', 'JFK', 'POU', '']
['MDH', 'MWA', '', '', '']
['BNA', '', '', '', '']
['SDF', '', '', '', '']
['DLH', '', '', '', '']
['', '', '', '', '']
['LBE', 'PIT', '', '', '']
['CCR', 'OAK', 'SFO', '', '']
['LGA', 'EWR', 'JFK', '', '']
['ATL', '', '', '', '']
['MDW', 'CGX', 'ORD', 'GYY', '']
['AUS', '', '', '', '']
['ABE', 'TTN', '', '', '']
['', '', '', '', '']
['DAY', '', '', '', '']
['DTW', '', '', '', '']
['CLE', '', '', '', '']
['BLV', '', '', '', '']
['BLV', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['MSP', '', '', '', '']
['POU', '', '', '', '']
['', '', '', '', '']
['SEA', '', '', '', '']
['SLC', '', '', '', '']
['TRI', '', '', '', '']
['DDC', '', '', '', '']
['', '', '', '', '']
['SLC', '', '', '', '']
['', '', '', 

...

['LFT', '', '', '', '']
['MOB', '', '', '', '']
['HGR', 'MDT', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['ALB', '', '', '', '']
['PIT', 'LBE', '', '', '']
['MSP', '', '', '', '']
['LBE', 'PIT', 'MGW', '', '']
['SLC', '', '', '', '']
['', '', '', '', '']
['HVN', '', '', '', '']
['TYS', '', '', '', '']
['TRI', '', '', '', '']
['DAY', '', '', '', '']
['LBE', 'PIT', '', '', '']
['BWG', '', '', '', '']
['MEM', '', '', '', '']
['LGA', 'POU', 'JFK', 'EWR', '']
['MDW', 'GYY', 'CGX', 'ORD', '']
['SLC', '', '', '', '']
['MEI', '', '', '', '']
['ORD', 'MDW', '', '', '']
['', '', '', '', '']
['CLT', '', '', '', '']
['PIT', 'YNG', '', '', '']
['', '', '', '', '']
['ORD', 'CGX', 'MDW', '', '']
['IAH', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['SEA', '', '', '', '']
['CHA', '', '', '', '']
['POU', 'LGA', '', '', '']
['', '', '', '', '']
['PHL', 'RDG', 'LNS', '', '']
['MKL', '', '', '', '']
['CKB', 'MGW', '', '', '']
['HSV', '', '', '', '']
['PHL', 'TTN', '', '', '']
['LG

['', '', '', '', '']
['DTW', 'FNT', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['YNG', '', '', '', '']
['TLH', '', '', '', '']
['GRR', 'AZO', 'LAN', '', '']
['', '', '', '', '']
['MCI', '', '', '', '']
['', '', '', '', '']
['AGS', '', '', '', '']
['LNK', '', '', '', '']
['LGA', 'EWR', 'JFK', 'POU', '']
['PHL', 'TTN', '', '', '']
['GEG', '', '', '', '']
['TUL', '', '', '', '']
['MKE', '', '', '', '']
['BHM', '', '', '', '']
['LGA', 'JFK', 'EWR', '', '']
['', '', '', '', '']
['FLO', '', '', '', '']
['ICT', '', '', '', '']
['', '', '', '', '']
['SMF', '', '', '', '']
['JBR', '', '', '', '']
['CLT', '', '', '', '']
['', '', '', '', '']
['AUS', '', '', '', '']
['MKE', '', '', '', '']
['SEA', '', '', '', '']
['CRW', '', '', '', '']
['', '', '', '', '']
['JAX', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['CHA', '', '', '', '']
['CLE', '', '', '', '']
['TRI', '', '', '', '']
['LGA', 'EWR', 'JFK', '', '']
['AOO', '', '', '', '']
['MKE', '', '', '', '']
['EAU', '', '', 

['ORD', '', '', '', '']
['DTW', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['SEA', '', '', '', '']
['YNG', 'PIT', '', '', '']
['ATL', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['GNV', '', '', '', '']
['', '', '', '', '']
['SNA', 'LGB', 'ONT', 'LAX', '']
['', '', '', '', '']
['', '', '', '', '']
['MSP', '', '', '', '']
['LWT', '', '', '', '']
['RAP', '', '', '', '']
['JAN', '', '', '', '']
['', '', '', '', '']
['PHX', '', '', '', '']
['MOB', '', '', '', '']
['', '', '', '', '']
['LGA', 'EWR', 'JFK', '', '']
['', '', '', '', '']
['LBB', '', '', '', '']
['TUP', '', '', '', '']
['SDF', '', '', '', '']
['JFK', 'EWR', 'LGA', '', '']
['JFK', 'EWR', '', '', '']
['DBQ', '', '', '', '']
['INL', '', '', '', '']
['AVP', '', '', '', '']
['AVP', '', '', '', '']
['ASE', 'EGE', '', '', '']
['PIA', '', '', '', '']
['MDW', 'CGX', 'ORD', 'GYY', '']
['EGE', 'ASE', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['LFT', '', '', 

['ROC', '', '', '', '']
['STS', '', '', '', '']
['TYR', '', '', '', '']
['PSP', '', '', '', '']
['ORD', 'MDW', '', '', '']
['MGM', '', '', '', '']
['', '', '', '', '']
['RDG', 'LNS', 'ABE', '', '']
['', '', '', '', '']
['CLE', '', '', '', '']
['', '', '', '', '']
['ITH', 'ELM', '', '', '']
['COS', '', '', '', '']
['RDG', 'LNS', '', '', '']
['DEC', '', '', '', '']
['RDG', '', '', '', '']
['CVG', '', '', '', '']
['LNS', 'RDG', 'MDT', '', '']
['SBA', 'SMX', '', '', '']
['', '', '', '', '']
['AVL', 'HKY', '', '', '']
['TPA', '', '', '', '']
['BLF', '', '', '', '']
['', '', '', '', '']
['BLV', '', '', '', '']
['PIT', 'LBE', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['WBU', 'DEN', '', '', '']
['FNT', '', '', '', '']
['JFK', '', '', '', '']
['', '', '', '', '']
['LGA', 'EWR', 'JFK', '', '']
['BUF', '', '', '', '']
['PHX', '', '', '', '']
['', '', '', '', '']
['LIT', '', '', '', '']
['CNY', '', '', '', '']
['', '', '', '', '']
['JFK', 'EWR', '', '', '']
['CGI', 'PAH', '', '', '']
[

['PIT', 'LBE', '', '', '']
['YNG', '', '', '', '']
['GSO', '', '', '', '']
['AGS', '', '', '', '']
['SJC', 'SFO', 'OAK', '', '']
['', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['AVL', '', '', '', '']
['', '', '', '', '']
['FLL', '', '', '', '']
['', '', '', '', '']
['MKL', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['DEC', '', '', '', '']
['EUG', 'SLE', '', '', '']
['', '', '', '', '']
['MCO', '', '', '', '']
['', '', '', '', '']
['ATL', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['ATL', '', '', '', '']
['', '', '', '', '']
['', '', '', '', '']
['CDC', '', '', '', '']
['', '', '', '', '']
['MKE', 'JVL', '', '', '']
['BLV', '', '', '', '']
['', '', '', '', '']
['BLV', '', '', '', '']
['', '', '', '', '']
['BLV', '', '', '', '']
['ABE', 'TTN', 'PHL', 'RDG', '']
['', '', '', '', '']
['FSM', '', '', '', '']
['', '', '', '', '']
['EAU', '', '', '', '']
['PHL', 'TTN', '', '', '']
['AEX', '', '', '', '']
['IND', '', '', '', '']
['MKL', '', '', '',

In [171]:
city_airport

,city_id,city_name,closest_airport,second_closest_airport,state,third_closest_airport
0,g60763,New York,LGA,EWR,NY,JFK
1,g32655,Los Angeles,LAX,BUR,CA,LGB
2,g35805,Chicago,GYY,CGX,IL,MDW
3,g56003,Houston,HOU,IAH,TX,
4,g31310,Phoenix,PHX,,AZ,
5,g60795,Philadelphia,PHL,TTN,PA,
6,g60956,San Antonio,SAT,,TX,
7,g55711,Dallas,,,TX,
8,g60750,San Diego,SAN,CLD,CA,
9,g33020,San Jose,SJC,,CA,
